In [9]:
import os
!ls
# Change to the cloned repo directory (optional)
!git clone "https://github.com/ezragershman/spotify-listening-data"
%cd '/content/spotify-listening-data'

for dirname, _, filenames in os.walk('/content/spotify-listening-data'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

sample_data
Cloning into 'spotify-listening-data'...
remote: Enumerating objects: 97, done.
remote: Counting objects: 100% (97/97), done.
remote: Compressing objects: 100% (77/77), done.
remote: Total 97 (delta 29), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (97/97), 2.10 MiB | 3.59 MiB/s, done.
Resolving deltas: 100% (29/29), done.
/content/spotify-listening-data
/content/spotify-listening-data/README.md
/content/spotify-listening-data/notebooks/data_analysis.ipynb
/content/spotify-listening-data/data/raw/READMEplaceholder.md
/content/spotify-listening-data/data/raw/Streaming_History_Audio_2013-2024.json
/content/spotify-listening-data/data/processed/spotify_data.csv
/content/spotify-listening-data/.git/HEAD
/content/spotify-listening-data/.git/description
/content/spotify-listening-data/.git/packed-refs
/content/spotify-listening-data/.git/index
/content/spotify-listening-data/.git/config
/content/spotify-listening-data/.git/objects/pack/pack-b4a4f34cedcba5e3a

In [20]:
!git pull --rebase

Successfully rebased and updated refs/heads/main.


In [13]:
%cd '/content/spotify-listening-data'
from google.colab import userdata
github_token = userdata.get('github_token')

!git config --global user.email "ezragersh@gmail.com"
!git config --global user.name "ezragershman"
# Set up Git to use the token for authentication
if github_token:
    repo_url = f"https://{github_token}@github.com/ezragershman/spotify-listening-data.git"  # Replace 'YourGitHubUsername' and 'YourRepository' with your actual GitHub username and repo
    !git remote set-url origin {repo_url}
    !git add *
    !git commit -m "Added geolocated ip data pt2"
    !git push origin main
else:
    print("GitHub token is not available.")

/content/spotify-listening-data
[main 8fa0d21] Added geolocated ip data pt2
 2 files changed, 168 insertions(+), 168 deletions(-)
 rewrite data/processed/deduplicated_unique_ips.csv (97%)
 rewrite data/processed/geolocated_ips.csv (100%)
Enumerating objects: 11, done.
Counting objects: 100% (11/11), done.
Delta compression using up to 2 threads
Compressing objects: 100% (6/6), done.
Writing objects: 100% (6/6), 1.52 KiB | 1.52 MiB/s, done.
Total 6 (delta 1), reused 0 (delta 0), pack-reused 0
remote: Resolving deltas: 100% (1/1), completed with 1 local object.
To https://github.com/ezragershman/spotify-listening-data.git
   c868ce6..8fa0d21  main -> main


# **MAIN FUNCTION TO UPDATE & SAVE MUSIC LIBRARY DATABASE**


In [ ]:
import requests
from base64 import b64encode
import os
from google.colab import userdata

# Step 1: Set up Spotify credentials as environment variables in Google Colab
CLIENT_ID = userdata.get('spotify_client_id')
CLIENT_SECRET = userdata.get('spotify_client_sec')

# Step 2: Get Access Token from Spotify API using Client Credentials Flow
def get_access_token(client_id, client_secret):
    auth_header = b64encode(f"{client_id}:{client_secret}".encode('utf-8')).decode('utf-8')
    headers = {
        'Authorization': f'Basic {auth_header}',
        'Content-Type': 'application/x-www-form-urlencoded'
    }
    data = {'grant_type': 'client_credentials'}

    response = requests.post('https://accounts.spotify.com/api/token', headers=headers, data=data)
    if response.status_code == 200:
        return response.json()['access_token']
    else:
        print("Failed to get access token")
        print(response.json())  # Added for better debugging
        return None

# Step 3: Search for a track by name and artist
def search_track(track_name, artist_name, access_token):
    headers = {'Authorization': f'Bearer {access_token}'}
    query = f"track:{track_name} artist:{artist_name}"
    params = {'q': query, 'type': 'track', 'limit': 1}

    response = requests.get('https://api.spotify.com/v1/search', headers=headers, params=params)
    if response.status_code == 200:
          track = response.json()
          return {
              'album_type': track['album']['album_type'],
              'track_uri': track['uri'],
              'explicit': track['explicit'],
              'available_markets': track['available_markets']
          }
        else:
            print("No track found")
            return None
    else:
        print("Failed to search for track")
        print(response.json())  # Added for better debugging
        return None


In [ ]:
# Step 4: Example usage - search for "Never Gonna Give You Up" by Rick Astley
access_token = get_access_token(CLIENT_ID, CLIENT_SECRET)
if access_token:
    track_info = search_track("Never Gonna Give You Up", "Rick Astley", access_token)
    if track_info:
        print(track_info)

{'Track Name': 'Never Gonna Give You Up', 'Artist': 'Rick Astley', 'Album': 'Whenever You Need Somebody', 'Release Date': '1987-11-12', 'Duration (ms)': 213573, 'Popularity': 78, 'URI': 'spotify:track:4PTG3Z6ehGkBFwjybzWkR8'}


# **MAIN FUNCTION TO UPDATE & SAVE TO DATABASE**
Set json_file_path to the name of the file you want to process.

In [7]:
json_file_path = "/content/spotify-listening-data/data/raw/Streaming_History_Audio_2013-2024.json"
json_to_csv(json_file_path)

Data successfully updated and saved to /content/spotify-listening-data/data/processed/spotify_data.csv


In [11]:
import pandas as pd
import uuid
import os
import requests

def json_to_csv(json_file_path, csv_output_path='/content/spotify-listening-data/data/processed/spotify_data.csv'):
    # Load the new JSON data into a DataFrame
    new_data = pd.read_json(json_file_path)

    # Rename columns for consistency
    new_data.rename(columns={
        'ts': 'end_time_UTC',
        'platform': 'platform',
        'ms_played': 'ms_played',
        'conn_country': 'connection_country',
        'ip_addr_decrypted': 'ip_address',
        'master_metadata_track_name': 'track_name',
        'master_metadata_album_artist_name': 'album_artist_name',
        'master_metadata_album_album_name': 'album_name',
        'spotify_track_uri': 'track_uri',
        'episode_name': 'episode_name',
        'episode_show_name': 'episode_show_name',
        'spotify_episode_uri': 'episode_uri',
        'reason_start': 'start_reason',
        'reason_end': 'end_reason',
        'shuffle': 'shuffle',
        'skipped': 'skipped',
    }, inplace=True)

    # # Get the location data and normalize it into separate columns
    # location_data = new_data['ip_address'].apply(lambda ip: get_location_from_ip(ip))

    # # Normalize the location data into separate columns
    # location_df = json_normalize(location_data)

    # # Merge the location data with the original DataFrame
    # new_data = pd.concat([new_data, location_df], axis=1)

    # Generate a unique ID for each row
    new_data['unique_id'] = new_data['end_time_UTC'].apply(lambda x: f"{x}_{uuid.uuid4().hex[:8]}")

    # Keep only the relevant columns for the CSV
    new_data = new_data[['unique_id', 'end_time_UTC', 'platform', 'ms_played', 'connection_country',
                         'ip_address', 'track_name', 'album_artist_name', 'album_name',
                         'track_uri', 'episode_name', 'episode_show_name', 'episode_uri',
                         'start_reason', 'end_reason', 'shuffle', 'skipped',
                         # 'country', 'region', 'city', 'latitude', 'longitude'
                         ]]

    # If the output CSV already exists, read it to check for duplicates
    if os.path.exists(csv_output_path):
        existing_data = pd.read_csv(csv_output_path)

        # Identify and keep only new rows that are not already in the CSV
        merged_data = pd.concat([existing_data, new_data]).drop_duplicates(subset=['unique_id'], keep='first')
    else:
        # If no existing CSV, just use new data
        merged_data = new_data

    # Ensure the 'processed' directory exists (create it if not)
    os.makedirs(os.path.dirname(csv_output_path), exist_ok=True)

    # Save updated data to CSV in the specified path
    merged_data.to_csv(csv_output_path, index=False)

    print(f"Data successfully updated and saved to {csv_output_path}")

ModuleNotFoundError: No module named 'ip2geotools'

In [12]:
import pandas as pd
# Create table for unique ID information.

def extract_unique_ips(json_file_path, output_csv_path='/content/spotify-listening-data/data/processed/unique_ips.csv'):
    # Load the JSON data into a DataFrame
    data = pd.read_json(json_file_path)

    # Extract the IP address column (adjust column name based on your data)
    ip_addresses = data['ip_addr_decrypted']  # Ensure 'ip_addr_decrypted' is the correct column name

    # Get the unique IP addresses
    unique_ips = ip_addresses.dropna().unique()

    # Create a DataFrame from the unique IP addresses
    unique_ips_df = pd.DataFrame(unique_ips, columns=['unique_ip_address'])

    # Save the unique IPs to a CSV file
    unique_ips_df.to_csv(output_csv_path, index=False)

    print(f"Unique IP addresses extracted and saved to {output_csv_path}")

# Example usage
json_file_path = '/content/spotify-listening-data/data/raw/Streaming_History_Audio_2013-2024.json'
extract_unique_ips(json_file_path)


Unique IP addresses extracted and saved to /content/spotify-listening-data/data/processed/unique_ips.csv


In [23]:
import pandas as pd
from ipaddress import ip_network, ip_address

# Load IP data into DataFrame (assuming your list is saved as 'unique_ips.csv')
df = pd.read_csv('data/processed/unique_ips.csv')

def group_by_subnet(ip):
    try:
        # If it's an IPv4 address
        ip_obj = ip_address(ip)
        if ip_obj.version == 4:
            return f"{ip_obj.exploded.split('.')[0]}.{ip_obj.exploded.split('.')[1]}.{ip_obj.exploded.split('.')[2]}.0/24"
        else:
            # For IPv6, return the first few blocks (customizable)
            return ':'.join(ip.split(':')[:4]) + '::/64'
    except ValueError:
        return 'Invalid IP'

# Apply subnet grouping
df['subnet'] = df['unique_ip_address'].apply(group_by_subnet)

# Drop duplicates based on subnets, keeping one representative IP
deduplicated_ip = df.drop_duplicates(subset='subnet')

# Save the deduplicated DataFrame as a CSV file
deduplicated_ip.to_csv('data/processed/deduplicated_unique_ips.csv', index=False)

In [12]:
import time
import pandas as pd
from ip2geotools.databases.noncommercial import DbIpCity

# Load previously processed IPs from CSV if it exists
def load_processed_ips(file_path):
    try:
        df = pd.read_csv(file_path)
        print(f"Loaded {len(df)} previously processed IPs from {file_path}.")
        return set(df['unique_ip_address'].tolist())  # Assuming the CSV has a column named 'unique_ip_address'
    except FileNotFoundError:
        print(f"No previous geolocated IPs found. Starting fresh.")
        return set()

# Save new processed IPs to the CSV
def save_processed_ips(file_path, processed_ips):
    df = pd.DataFrame(list(processed_ips), columns=['unique_ip_address'])
    df.to_csv(file_path, index=False)
    print(f"Saved {len(processed_ips)} processed IPs to {file_path}.")

# Path to the CSV files
deduplicated_ips_file = '/content/spotify-listening-data/data/processed/deduplicated_unique_ips.csv'
geolocated_ips_file = '/content/spotify-listening-data/data/processed/geolocated_ips.csv'

# Load the IPs
print("Loading deduplicated IPs...")
deduplicated_df = pd.read_csv(deduplicated_ips_file)
print(f"Loaded {len(deduplicated_df)} deduplicated IPs from {deduplicated_ips_file}.")

processed_ips = load_processed_ips(geolocated_ips_file)

# Function to get location details from IP address
def get_location_from_ip(ip_address):
    try:
        # Query the ip2geotools database with the given IP address
        res = DbIpCity.get(ip_address, api_key="free")

        # Structure the location data
        location = {
            "city": res.city,
            "region": res.region,
            "country": res.country,
            "latitude": res.latitude,
            "longitude": res.longitude
        }
        return location
    except Exception as e:
        # Handle any potential errors
        return {
            "city": None,
            "region": None,
            "country": None,
            "latitude": None,
            "longitude": None,
            "error": f"Error: {str(e)}"
        }

# Add columns to store location data if not already present
if 'city' not in deduplicated_df.columns:
    print("Adding location columns to deduplicated IP data...")
    deduplicated_df['city'] = None
    deduplicated_df['region'] = None
    deduplicated_df['country'] = None
    deduplicated_df['latitude'] = None
    deduplicated_df['longitude'] = None

# Start processing IPs
print("Starting IP geolocation processing...")

for idx, ip in enumerate(deduplicated_df['unique_ip_address']):
    print(ip)
    # Check if this IP has already been processed
    if ip not in processed_ips:
        # Get location data for unprocessed IPs
        print(f"Processing IP {idx + 1}/{len(deduplicated_df)}: {ip}")
        location_data = get_location_from_ip(ip)
        deduplicated_df.at[idx, 'city'] = location_data['city']
        deduplicated_df.at[idx, 'region'] = location_data['region']
        deduplicated_df.at[idx, 'country'] = location_data['country']
        deduplicated_df.at[idx, 'latitude'] = location_data['latitude']
        deduplicated_df.at[idx, 'longitude'] = location_data['longitude']

        # Add the IP to the processed list
        processed_ips.add(ip)

        # Log the geolocation
        print(f"Processed {ip}: {location_data['city']}, {location_data['region']}, {location_data['country']}")

    # Wait 10 seconds to avoid hitting rate limits
    time.sleep(10)

# Save the updated DataFrame back to the original CSV
print("Saving updated deduplicated IPs to CSV...")
deduplicated_df.to_csv(deduplicated_ips_file, index=False)

# Save the processed IPs to the geolocated file
save_processed_ips(geolocated_ips_file, processed_ips)

print("Geolocation data added and saved successfully.")



Loading deduplicated IPs...
Loaded 83 deduplicated IPs from /content/spotify-listening-data/data/processed/deduplicated_unique_ips.csv.
Loaded 83 previously processed IPs from /content/spotify-listening-data/data/processed/geolocated_ips.csv.
Adding location columns to deduplicated IP data...
Starting IP geolocation processing...
173.79.66.78
70.110.22.156
108.18.101.195
71.246.200.163
129.64.121.98
129.64.166.4
129.64.123.114
129.64.155.124
129.64.153.87
107.127.45.66
24.148.79.81
107.119.44.1
166.194.135.178
166.205.140.99
71.114.105.10
199.195.178.243
108.147.92.132
2600:387:f:5b17::2
2600:387:f:7619::9
2600:387:f:6a16::b
166.199.152.13
96.95.91.5
2600:387:f:5b14::4
166.194.132.52
2600:387:15:2c1b::5
2600:387:f:6a17::7
2600:387:f:5b13::8
2600:387:f:7618::c
2600:387:f:6a1b::7
107.77.206.136
2600:387:f:6a10::4
12.74.213.82
2600:387:15:2c10::a
2601:249:8680:3250:78e4:194:60c:7ff2
24.13.173.197
2600:387:f:5b19::7
2600:387:f:5b10::5
2600:387:f:6a18::3
2600:387:f:5b15::2
2600:387:15:2c11: